# NLP4datascience Module - Tutorial 1: Text Preprocessing
This python module's purpose is to facilitate the use of natural language inputs (ie. text) in data science and data analysis. For the foundational implementaiton, it makes use of popular NLP elements in the python modules NLTK, Sci-Kit Learn and Gensim and offers further proprietary functionalities beyond those. 

## 1. Preprocessing data

#### Import modules

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse
from tqdm import tqdm
from nlp4datascience.datahandling.largepickle import pickle_load, pickle_dump
from nlp4datascience.preprocessors.nlp4datascience import BagOfWords
from nlp4datascience.preprocessors.nlp4datascience import DTM
from gensim.models.phrases import Phrases, Phraser


#### Load data

In [4]:
text = "<pathname>"

#### Text data preprocesing
BagOfWords creates an nlp4datascience-object which lets us preprocess and tokenize our text data with a few simply commands.
min_length defines the minimum character length of words being considered. Words with fewer characters are automatically being deleted.
ngram_length defines the the degreee of ngrams we want to extract. Set this to 1 for now. We will extract bigrams, trigrams, ngrams later via a collocation approach.

In [ ]:
preprocessed_text = BagOfWords(text, min_length=1, ngram_length=1)

We now have our NLP4datascience object, called "preprocessed_text".
Let's clean the text by:
1. removing stopwords (based on NLTK stopword list + our own list)
2. lowercasing the corpus
3. removing punctuation
4. removing special characters (can be customized)
5. removing numbers (can be switched off)

If we want to add our own stopwords to the stopword list, then add the words by using the following line of code or upload a list of words and then as preprocessed_text.custom_stopwords.

In [ ]:
preprocessed_text.custom_stopwords = ["my_stopword+1","my_stopword_2",...,"my_stopword_N"]

Let's clean and tokeinze our text corpus

In [ ]:
preprocessed_text.clean()
preprocessed_text.tokenize()

Let's create unigrams via stemming (we could also use lemmatization instead).

In [ ]:
preprocessed_text.stemm() # use preprocessed_text.lemmatize() if you want to use this approach instead

Let's visually inspect the preprocessed corpus. The graph plots the ranking of the ngrams either according to a tf, df or a tf-idf weighting. Let's choose the popular tf-idf ranking.

In [ ]:
preprocessed_text.visualize(weight = "tf-idf")

If we feel we want to cut the corpus at a certain minimum threshold of tf-idf scores, we can do this by using the following command

In [ ]:
preprocessed_text.rank_remove(rank="tf-idf", items="uni", cutoff = 15)

Let's visually inspect our corpus after applying the tf-idf cutoff

preprocessed_text.visualize(weight = "tf-idf")

We could even apply another cutoff criterion, say based on document frequency, on top of the former one.

Once we are happy with our results, we can save them to our computer. You can choose to save it as a pickle file (.pkl) or a csv file (.csv). The preprocessing is now completed.

In [ ]:
preprocessed_text.save("pkl", <path_where_to_save_the_file", str("<filename>"))

## 2. Create a Document-Term-Matrix (based on unigrams)

Load the preprocessed text corpus from step 1

In [ ]:
ngrams = pickle_load("<preprocessed_data_from_step_1>.pkl")


Create a DTM object based on the NLP4datascience module.

In [ ]:
dtm_object = DTM(ngrams.uni_adj)

Create the DTM based on unigrams and inspect tokens

In [ ]:
dtm_object.unigram_dtm()
dtm_object.dtm
dtm_object.tokens

For bigrams, we specify the minimum frequency of bigrams (and unigrams in them) occuring as well as a scoring-function threshold. The higher the threshold, the fewer bigrams are formed (this is based on the collocation approach, and on the gensim implementation of it).

In [ ]:
dtm_object.bigram_dtm(min_count=10, threshold=10)
dtm_object.dtm
dtm_object.tokens

For ngrams (more specifically, up to 4-grams), we can use the following command

In [ ]:
dtm_object.ngram_dtm(min_count=10, threshold=10)
dtm_object.dtm
dtm_object.tokens

To save any of the DTMs and the corresponding vocabulary list to your computer, use the following commands

In [ ]:
scipy.sparse.save_npz('<path_where_to_store_the_compressed_dtm>.npz', dtm_object.dtm)
pd.Series(dtm_object.tokens).to_pickle('<path_where_to_save_the_file>.pkl')

If we want to inspect the vocabulary, we can run the following command

###### Note: 
The DTM is stored as a Compressed Sparse Row (CSR) scipy matrix. This is advantageous when the corpus is getting too large to be fully held in memory in an uncompressed form, for example, when we have millions of documents and (hundred) thousands of unique ngram terms.

If you want to bring your compressed DTM into a non-compressed (ie. dense) format, use the below command

In [ ]:
dtm_dense = dtm_sparse.todense()